# Build Fundamentals

In [15]:
import wrds
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
pd.set_option("display.max_columns", None)

In [2]:
prc_stmt = """
SELECT
    prc.gvkey,
    prc.datadate AS date,
    cshoc,
    prccd
FROM ( SELECT DISTINCT
        gvkey,
        iid
    FROM
        comp_global_daily.g_funda
    WHERE
        exchg = ANY (ARRAY [104, 132, 151, 154, 171, 172, 192, 194, 201, 209, 228, 256, 257, 273, 286])
        AND curcd = 'EUR') AS fund
    JOIN comp_global_daily.g_sec_dprc AS prc ON fund.gvkey = prc.gvkey
        AND fund.iid = prc.iid
WHERE
    curcdd = 'EUR'
    AND cshtrd IS NOT NULL
    AND datadate >= '1999-01-01'
    AND prc.gvkey = '100022'
"""

In [90]:
fund_stmt = """
SELECT
    gvkey,
    datadate AS date,
    loc AS country,
    indfmt AS industry,
    sicdesc AS classification,
    act,
    at AS att,
    ap,
    capx,
    ceq,
    ch,
    che,
    chee, 
    cogs,
    dlc,
    dltis,
    dltr,
    dltt,
    dp,
    dv,
    dvc,
    dvt,
    dlcch,
    ebit,
    ebitda,
    -- COALESCE(epsexcon, nicon / cshpria) AS eps,
    -- COALESCE(epsincon, (nicon + xido) / cshpria) AS epsin,
    fincf,
    gdwl,
    icapt,
    intan,
    invt,
    ivaeq,
    ivao,
    ivncf,
    ivst,
    lct,
    lt AS ltt,
    mib,
    nicon,
    oancf,
    pi,
    ppegt,
    ppent,
    pstk,
    prstkc,
    COALESCE(rect, artfs) AS rect,
    revt,
    COALESCE(seq, ceq + pstk, at - lt) AS seq,
    sstk,
    teq,
    COALESCE(txp, txpfs) AS txp,
    xint,
    xrd,
    xsga
FROM
    comp.g_funda AS fund
    JOIN comp.r_siccd AS sic ON fund.sich::VARCHAR = sic.siccd
WHERE
    exchg = ANY (ARRAY [104, 107, 132, 151, 154, 171, 192, 194, 201, 209, 256, 257, 273, 276, 286])
    AND curcd = 'EUR'
    AND gvkey = '100022'
ORDER BY
    gvkey, datadate;
"""

In [83]:
def query_wrds(sql_stmt):
    with wrds.Connection(wrds_username="iewaij") as db:
        data = db.raw_sql(
            sql_stmt,
            date_cols=["date", "datadate"],
        )
    return data

In [84]:
def query_fundamental():
    fund = (
        query_wrds(fund_stmt)
        .fillna(0)
        .astype(
            {
                "gvkey": "object",
                "country": "category",
                "industry": "category",
                "classification": "category",
            }
        )
    )
    return fund

In [85]:
def query_price():
    prc = query_wrds(prc_stmt)
    return prc

In [86]:
def fill_year(df):
    first_date = df["date"].iloc[0]
    last_date = df["date"].iloc[-1]
    date_index = pd.date_range(
        pd.to_datetime(first_date), pd.to_datetime(last_date) + DateOffset(years=1), name="date"
    )
    return (
        df.drop("gvkey", axis=1)
        .set_index("date")
        .sort_index()
        .reindex(date_index, method="ffill")
    )

In [91]:
fund = query_fundamental()

Loading library list...
Done


In [92]:
def build_fundamental(df):
    oa = df.att - df.che
    ol = df.att - df.dlc - df.dltt - df.mib - df.pstk - df.ceq
    chact = df.act - df.act.shift(1)
    chchee = df.chee - df.chee.shift(1)
    chlct = df.lct - df.lct.shift(1)
    chdlc = df.dlc - df.dlc.shift(1)
    chtxp = df.txp - df.txp.shift(1)
    chchee = df.chee - df.chee.shift(1)
    avg_at = (df.att + df.att.shift(1)) / 2
    nca = df.att - df.act - df.ivaeq
    ncl = df.ltt - df.lct - df.dltt
    ncoa = nca - ncl
    coa = df.act - df.che
    col = df.lct - df.dlc
    wc = df.act - df.che - df.lct + df.dlc
    fna = df.ivst + df.ivao
    fnl = df.dltt + df.dlc + df.pstk
    nfna = fna - fnl
    be = df.seq - df.pstk
    df = df.assign(
        # Accruals
        accruals_acc=((chact - chchee) - (chlct - chdlc - chtxp) - df.dp) / avg_at,
        accruals_chcoa=(coa - coa.shift(1)) / df.att.shift(1),
        accruals_chcol=(col - col.shift(1)) / df.att.shift(1),
        accruals_chnncwc=(wc - wc.shift(1)) / df.att.shift(1),
        accruals_chnncoa=(ncoa - ncoa.shift(1)) / df.att.shift(1),
        accruals_chncoa=(nca - nca.shift(1)) / df.att.shift(1),
        accruals_chncol=(ncl - ncl.shift(1)) / df.att.shift(1),
        accruals_chnfa=nfna - nfna.shift(1) / df.att.shift(1),
        accruals_chlti=(df.ivao - df.ivao.shift(1)) / df.att.shift(1),
        accruals_chce=(df.ceq - df.ceq.shift(1)) / df.att.shift(1),
        accruals_chfl=(df.dltt + df.dlc + df.pstk - (df.dltt + df.dlc + df.pstk).shift(1)) / df.att.shift(1),
        accruals_grii=(df.invt - df.invt.shift(1)) / ((df.att + df.att.shift(1)) / 2),
        accruals_ich=(df.invt - df.invt.shift(1)) / df.att.shift(1),
        accruals_igr=(df.invt - df.invt.shift(1)) / df.invt.shift(1),
        accruals_nwcch=(wc - wc.shift(1)) / df.att.shift(1),
        accruals_poa = (df.nicon - df.oancf) / abs(df.nicon),
        accruals_pta =(df.nicon - (- df.sstk + df.prstkc + df.dv + df.oancf + df.ivncf + df.fincf)) / abs(df.nicon),
        accruals_ta=((ncoa + wc + nfna) - (ncoa + wc + nfna).shift(1)) / df.att.shift(1),
        # Efficiency
        efficiency_itr = df.cogs / df.invt,
        efficiency_rtr = df.revt / df.rect,
        efficiency_apr = df.cogs / df.ap,
        efficiency_dsi = 365 * df.invt / df.cogs,
        efficiency_dso = 365 * df.rect / df.revt,
        efficiency_dpo = 365 * df.ap / df.cogs,
        # Profitablity
        profitability_at = df.revt / ((oa - ol) + (oa - ol).shift(1))/2,
        profitability_fat = df.revt / df.ppent,
        profitability_ct = df.revt / df.att.shift(1),
        profitability_gp = (df.revt - df.cogs) / df.att.shift(1),
        profitability_opta = (df.revt - df.cogs - df.xsga + df.xrd) / df.att,
        profitability_opte = (df.revt - df.cogs - df.xsga + df.xrd) / be,
        profitability_gpm = (df.revt - df.cogs) / df.revt,
        profitability_ebitdam = df.ebitda / df.revt,
        profitability_ebitm = df.ebit / df.revt,
        profitability_ptm = df.pi / df.revt,
        profitability_npm = df.nicon / df.revt,
        profitability_roa = df.nicon / df.att,
        profitability_roe = df.nicon / be,
        profitability_roic = (df.ebit * (df.nicon / df.pi)) / (df.dlc + df.dltt + df.teq),
        # Investment
        investment_agr=df.att / df.att.shift(1),
        investment_cdi=np.log((df.dltt + df.dlc) / (df.dltt.shift(5) + df.dlc.shift(5))),
        investment_chnoa=(((oa - ol) / df.att.shift(1)) - (((oa - ol) / df.att.shift(1)).shift(1)))
        / df.att.shift(1),
        investment_chppeia=((df.ppegt - df.ppegt.shift(1)) + (df.invt - df.invt.shift(1)))
        / df.att.shift(1),
        investment_griltnoa=(
            ((oa - ol) / df.att.shift(1))
            - (((oa - ol) / df.att.shift(1)).shift(1))
            - ((chact - chchee) - (chlct - chdlc - chtxp) - df.dp) / avg_at
        ),
        investment_inv=(df.capx / df.revt)
        / (
            (
                (df.capx.shift(1) / df.revt.shift(1))
                + (df.capx.shift(2) / df.revt.shift(2))
                + (df.capx.shift(3) / df.revt.shift(3))
            )
            / 3
        ),
        investment_ndf=(df.dltis - df.dltr + df.dlcch) / ((df.att + df.att.shift(1)) / 2),
        investment_nef=(df.sstk - df.prstkc - df.dv) / ((df.att + df.att.shift(1)) / 2),
        investment_noa=(oa - ol) / df.att.shift(1),
        investment_noach=(ncoa - ncoa.shift(1)) / df.att,
        investment_txfin=(df.sstk - df.dv - df.prstkc + df.dltis - df.dltr) / df.att,
        # Leverage
        leverage_de = (df.dlc + df.dltt) / be,
        leverage_da = (df.dltt + df.dlc) / df.att,
        leverage_fl = df.att / be,
        leverage_deda = (df.dltt + df.dlc) / df.ebitda,
        leverage_ndeda = (df.dltt + df.dlc - df.chee) / df.ebitda,
        leverage_eic = df.ebit / df.xint,
        leverage_edaic = df.ebitda / df.xint,
        leverage_cac = df.ch / df.xint,
        leverage_dcap = (df.dltt + df.dlc) / (df.dltt + df.dlc + df.teq),
        leverage_cad = df.oancf / (df.dlc + df.dltt),
        # Liquidity
        liquid_cur = df.act / df.lct,
        liquid_qur = (df.act - df.invt) / df.lct,
        liquid_car = df.chee / df.lct,
        liquid_opr = df.oancf / df.lct,
        liquid_capxr = df.capx / df.oancf,
        # Other
        other_ir=(df.icapt - df.icapt.shift(1)) / (df.ebit * (df.nicon / df.pi)),
        other_nopat_g=(df.icapt - df.icapt.shift(1)) / df.icapt,
        other_rev_cagr_3=((df.revt / df.revt.shift(3)) ** (1 / 3)) - 1,
        other_ebitda_cagr_3=((df.ebitda / df.ebitda.shift(3)) ** (1 / 3)) - 1,
    )
    return df

In [93]:
fund_build = fund.groupby("gvkey").apply(build_fundamental)
fund_build

,gvkey,date,country,industry,classification,act,att,ap,capx,ceq,ch,che,chee,cogs,dlc,dltis,dltr,dltt,dp,dv,dvc,dvt,dlcch,ebit,ebitda,fincf,gdwl,icapt,intan,invt,ivaeq,ivao,ivncf,ivst,lct,ltt,mib,nicon,oancf,pi,ppegt,ppent,pstk,prstkc,rect,revt,seq,sstk,teq,txp,xint,xrd,xsga,accruals_acc,accruals_chcoa,accruals_chcol,accruals_chnncwc,accruals_chnncoa,accruals_chncoa,accruals_chncol,accruals_chnfa,accruals_chlti,accruals_chce,accruals_chfl,accruals_grii,accruals_ich,accruals_igr,accruals_nwcch,accruals_poa,accruals_pta,accruals_ta,efficiency_itr,efficiency_rtr,efficiency_apr,efficiency_dsi,efficiency_dso,efficiency_dpo,profitability_at,profitability_fat,profitability_ct,profitability_gp,profitability_opta,profitability_opte,profitability_gpm,profitability_ebitdam,profitability_ebitm,profitability_ptm,profitability_npm,profitability_roa,profitability_roe,profitability_roic,investment_agr,investment_cdi,investment_chnoa,investment_chppeia,investment_griltnoa,investment_inv,investment_ndf,investment_nef,investment_noa,investment_noach,investment_txfin,leverage_de,leverage_da,leverage_fl,leverage_deda,leverage_ndeda,leverage_eic,leverage_edaic,leverage_cac,leverage_dcap,leverage_cad,liquid_cur,liquid_qur,liquid_car,liquid_opr,liquid_capxr,other_ir,other_nopat_g,other_rev_cagr_3,other_ebitda_cagr_3
0,100022,1999-12-31,DEU,INDL,MOTOR VEHICLES AND PASSENGER CAR BODIES,28736.0,37507.0,3679.0,2125.0,3883.0,2055.0,2939.0,2055.0,26715.0,7379.0,0,0,8083.0,2042.0,234.0,0.0,0.0,0.0,931.0,2973.0,2301.0,0.0,12015.0,75.0,3621.0,185.0,710.0,-7594.0,884.0,13211.0,33575.0,0.0,0.0,5300.0,1111.0,23489.0,7801.0,49.0,0.0,13206.0,34402.0,3932.0,19.0,0.0,297.0,1483.0,0.0,4700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,-inf,NaN,7.377796,2.605028,7.261484,49.472768,140.113656,50.265207,NaN,4.409947,NaN,NaN,0.079638,0.769251,0.223446,0.086419,0.027062,0.032295,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005732,3.981973,0.412243,9.659284,5.200807,4.509586,0.627782,2.004720,1.385705,1.000000,0.342776,2.175157,1.901067,0.155552,0.401181,0.400943,NaN,NaN,NaN,NaN
1,100022,2000-12-31,DEU,INDL,MOTOR VEHICLES AND PASSENGER CAR BODIES,29112.0,35875.0,3998.0,2313.0,4846.0,2879.0,3630.0,2879.0,26837.0,6803.0,0,0,7485.0,2137.0,269.0,286.0,309.0,0.0,1578.0,3715.0,-942.0,0.0,12381.0,103.0,2809.0,192.0,758.0,-3045.0,751.0,14017.0,30979.0,0.0,0.0,4778.0,1663.0,19477.0,5710.0,50.0,0.0,12392.0,35356.0,4896.0,22.0,0.0,373.0,1861.0,0.0,4714.0,-0.106048,-0.008398,0.036846,-0.045245,0.021036,-0.053723,-0.074759,-12828.628949,0.001280,0.025675,-0.031274,-0.022131,-0.021649,-0.224247,-0.045245,-inf,-inf,0.004799,9.553934,2.853131,6.712606,38.204158,127.929630,54.375303,0.552282,6.191944,0.942651,0.227131,0.106063,0.785184,0.240949,0.105074,0.044632,0.047036,0.000000,0.000000,0.000000,0.000000,0.956488,NaN,NaN,-0.128616,NaN,NaN,0.000000,-0.006732,0.414696,0.021993,-0.006885,2.948411,0.398272,7.403013,3.846030,3.071063,0.847931,1.996239,1.547018,1.000000,0.334406,2.076907,1.876507,0.205393,0.340872,0.484094,inf,0.029561,NaN,NaN
2,100022,2001-12-31,DEU,INDL,MOTOR VEHICLES AND PASSENGER CAR BODIES,32791.0,51259.0,2990.0,8047.0,10719.0,2437.0,3344.0,2437.0,25892.0,11720.0,0,0,13945.0,2835.0,310.0,0.0,0.0,0.0,3356.0,6191.0,-258.0,0.0,24715.0,143.0,4501.0,134.0,652.0,-6079.0,907.0,18286.0,40489.0,0.0,1728.0,5892.0,3242.0,30423.0,15263.0,51.0,0.0,23741.0,38463.0,10770.0,24.0,0.0,194.0,781.0,1663.0,4647.0,0.040283,0.110523,-0.018063,0.128585,0.361868,0.327889,-0.033979,-24156.642397,-0.002955,0.163707,0.317157,0.038837,0.047164,0.602350,0.128585,-2.409722,1.092014,0.174690,5.752499,1.620109,8.659532,63.450680,225.293529,42.150085,0.395344,2.520016,1.072139,0.350411,0.187031,0.894393,0.326834,0.160960,0.087253,0.084289,0.044926,0.033711,0.161209,0.069697,1.428822,NaN,1.415195e-05,0.352279,0.467419,NaN,0.000000,-0.006565,0.922397,0.253263,-0.005580,2.394346,0.500693,4.782069,4.145534,3.751898,4.297055,7.927017,3.120359,1.000000,0.229

In [8]:
prc = query_price()

Loading library list...
Done


In [12]:
prc_build = prc.set_index(["gvkey", "date"])

In [20]:
df = fund_build.join(prc_build)

In [31]:
#### Dividend yield ratio
dyr = ((df.dvc*10**6) / df.cshoc) / df.prccd
dyr
# Recheck here if dvt should be taken - at least for this example, only cash paid div (dv) has figures

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.000000
        2000-01-04    0.000000
                        ...   
        2020-12-27         NaN
        2020-12-28    0.037206
        2020-12-29    0.037373
        2020-12-30    0.037855
        2020-12-31         NaN
Length: 7672, dtype: float64

In [22]:
#### Price to earnings
pe = (df.cshoc * df.prccd) / (df.nicon*10**6)
pe

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03         inf
        2000-01-04         inf
                        ...   
        2020-12-27         NaN
        2020-12-28    9.082453
        2020-12-29    9.041669
        2020-12-30    8.926732
        2020-12-31         NaN
Length: 7672, dtype: float64

In [23]:
#### Price to cash flow
pocf = (df.cshoc * df.prccd) / (df.oancf*10**6)
pocf

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03     3.416383
        2000-01-04     3.310722
                        ...    
        2020-12-27          NaN
        2020-12-28    12.081001
        2020-12-29    12.026753
        2020-12-30    11.873870
        2020-12-31          NaN
Length: 7672, dtype: float64

In [24]:
#### Price to sales
psales = (df.cshoc * df.prccd) / (df.revt*10**6)
psales

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.526331
        2000-01-04    0.510053
                        ...   
        2020-12-27         NaN
        2020-12-28    0.424533
        2020-12-29    0.422627
        2020-12-30    0.417255
        2020-12-31         NaN
Length: 7672, dtype: float64

In [25]:
#### PEG
peg = (df.prccd/((df.nicon*10**6)/df.cshoc)) / ((((df.nicon*10**6)/df.cshoc)/(((df.nicon*10**6)/df.cshoc).shift(1)))-1)
peg

gvkey   date      
100022  1999-12-31    NaN
        2000-01-01    NaN
        2000-01-02    NaN
        2000-01-03    NaN
        2000-01-04    NaN
                     ... 
        2020-12-27    NaN
        2020-12-28    NaN
        2020-12-29    inf
        2020-12-30    inf
        2020-12-31    NaN
Length: 7672, dtype: float64

In [26]:
#### M/B ratio
mb = (df.cshoc * df.prccd) / (df.ceq*10**6)
mb

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    4.663104
        2000-01-04    4.518884
                        ...   
        2020-12-27         NaN
        2020-12-28    0.745746
        2020-12-29    0.742397
        2020-12-30    0.732960
        2020-12-31         NaN
Length: 7672, dtype: float64

In [27]:
#### EV Sales
evsales = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.revt*10**6)
evsales

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.917471
        2000-01-04    0.901193
                        ...   
        2020-12-27         NaN
        2020-12-28    1.409161
        2020-12-29    1.407254
        2020-12-30    1.401882
        2020-12-31         NaN
Length: 7672, dtype: float64

In [28]:
#### EV EBITDA
evebitda = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.ebitda*10**6)
evebitda

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    10.616493
        2000-01-04    10.428129
                        ...    
        2020-12-27          NaN
        2020-12-28     9.556102
        2020-12-29     9.543175
        2020-12-30     9.506743
        2020-12-31          NaN
Length: 7672, dtype: float64

In [30]:
#### EV EBIT
evebit = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / (df.ebit*10**6)
evebit

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    33.902076
        2000-01-04    33.300566
                        ...    
        2020-12-27          NaN
        2020-12-28    13.329275
        2020-12-29    13.311243
        2020-12-30    13.260426
        2020-12-31          NaN
Length: 7672, dtype: float64

In [31]:
#### EV EBITDA-Capex
evebitdacpx = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / ((df.ebitda-df.capx)*10**6)
evebitdacpx

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03    37.220321
        2000-01-04    36.559938
                        ...    
        2020-12-27          NaN
        2020-12-28    17.347741
        2020-12-29    17.324273
        2020-12-30    17.258135
        2020-12-31          NaN
Length: 7672, dtype: float64

In [32]:
#### EV Operating CF
evocf = ((df.cshoc * df.prccd) + (df.dlc + df.dltt + df.pstk + df.mib - df.chee)*10**6) / ((df.oancf)*10**6)
evocf

gvkey   date      
100022  1999-12-31          NaN
        2000-01-01          NaN
        2000-01-02          NaN
        2000-01-03     5.955251
        2000-01-04     5.849590
                        ...    
        2020-12-27          NaN
        2020-12-28    40.100663
        2020-12-29    40.046414
        2020-12-30    39.893532
        2020-12-31          NaN
Length: 7672, dtype: float64

In [33]:
#### R&D to Market Capitalization
rdmarket = (df.xrd*10**6) / (df.cshoc * df.prccd)
rdmarket

gvkey   date      
100022  1999-12-31         NaN
        2000-01-01         NaN
        2000-01-02         NaN
        2000-01-03    0.000000
        2000-01-04    0.000000
                        ...   
        2020-12-27         NaN
        2020-12-28    0.134537
        2020-12-29    0.135144
        2020-12-30    0.136884
        2020-12-31         NaN
Length: 7672, dtype: float64

In [34]:
#### Degree of Operating Leverage
dopl = ((df.ebit/df.ebit.shift(1))-1) / ((df.revt/df.revt.shift(1))-1)
dopl

gvkey   date      
100022  1999-12-31   NaN
        2000-01-01   NaN
        2000-01-02   NaN
        2000-01-03   NaN
        2000-01-04   NaN
                      ..
        2020-12-27   NaN
        2020-12-28   NaN
        2020-12-29   NaN
        2020-12-30   NaN
        2020-12-31   NaN
Length: 7672, dtype: float64